In [ ]:
# Setup Folders, Tokens, and Dependencies
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
from pathlib import Path
from klimainsights.environment.settings import MAPBOX_TOKEN

datasets_folder = Path('./klimainsights/data')
px.set_mapbox_access_token(MAPBOX_TOKEN)

# Loading the Data

In [ ]:
temperature_gdf = gpd.read_file(datasets_folder / "temperature.geojson")
biodiversity_gdf =  gpd.read_file(datasets_folder / 'biodiversity.geojson')
disaster_gdf = gpd.read_file(datasets_folder / 'disaster.geojson')

**APP IDEAS**
- The initial page will an animated of a short text on the center that catches the viewers' attention. It can be a call to action or a hook to incite curiosity.
- Below the Text will be a button to "Click to Learn More" or "Continue", leading to the Temperature page
- All pages will contain a header/nav that contains an offcanvas for easier navigation between pages

# Temperature

In [ ]:
print(temperature_gdf.shape)
temperature_gdf.head()

In [ ]:
def update_bar_fig(island_value):
    selected_gdf = temperature_gdf[(temperature_gdf['island_group'].isin(['Luzon']) == True)].drop(columns=['geometry'])
    island_gdf = pd.melt(selected_gdf, id_vars=['name', 'admin_div', 'island_group', 'Region'], var_name='decade', value_name='value')
    # Create the Figure with horizontal orientation
    bar_fig = px.bar(island_gdf, y='name', x='value', animation_frame="decade", orientation='h')
    bar_fig.update_layout(
        height=750,
        title=f'Average Temperature per Province<br>in {island_value}',
        margin=dict(l=20, r=20, t=75, b=50),
        yaxis=dict(
            title="Province Name",
            tickfont=dict(size=11)  # Adjust tick font size to prevent overlap
        ),
        xaxis=dict(
            title="Temperature (°C)",
            range=[0, 35],
            tickmode='linear',
            dtick=5,
            tickfont=dict(size=13),  # Adjust tick font size to prevent overlap
            tickangle=0
        ),
        font=dict(  # Adjust font size for main title
            size=12
        )
    )
    hover_template = "Average Temperature in<br>" + \
                     "<b>%{y}</b><br>" + \
                     "during the %{customdata[0]}:<br>" + \
                     "%{customdata[1]:.2f}°C"
    bar_fig.update_traces(hovertemplate=hover_template,
                           customdata=island_gdf[['decade', 'value']])

    for frame in bar_fig.frames:
        frame.data[0].hovertemplate = hover_template
        frame.data[0].customdata = island_gdf[['decade', 'value']]
    bar_fig.update_layout(
        updatemenus=[{
            'direction': 'left',
            'pad': {'t': 10, 'b': 10, 'l': 10, 'r': 10},
            'showactive': False,
            'type': 'buttons',
            'x': 0.06,
            'xanchor': 'right',
            'y': -0.14,
            'yanchor': 'top'
        }],
        sliders=[{
            'active': 0,
            'x': 0.98,
            'y': -0.08,
            'xanchor': 'right',
            'yanchor': 'top',
            'transition': {'duration': 300, 'easing': 'cubic-in-out'},
            'pad': {'t': 10, 'b': 10, 'l': 10, 'r': 10},
            'currentvalue': {
                'font': {'size': 15},
                'prefix': 'Decade:',
                'visible': True,
                'xanchor': 'right',
            },
            'visible': True,
        }]
    )

    highest_temp_1960 = island_gdf[island_gdf['decade'] == '1960s']['value'].max()  # Highest temperature for the decade 1960
    bar_fig.add_shape(  # Line representing highest temperature in the 1960s
        type="line",
        x0=highest_temp_1960,
        y0=-1,
        x1=highest_temp_1960,
        y1=island_gdf.name.nunique()-0.5,
        line=dict(
            color="red",
            width=1,
            dash="dash",
        ),
    )

    bar_fig.add_annotation(
        x=highest_temp_1960 + 1,
        y=island_gdf.name.nunique() / 1.3,  # Adjust the y position of the label
        text="Highest Temp in the 1960s",
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=0,  # Adjust the arrow position
        font=dict(
            color="red",
            size=12
        ),
        align="left",
        textangle=90  # Tilt the text 90 degrees
    )

    return bar_fig

bar_fig = update_bar_fig('Luzon')
bar_fig.show()

### TEMP 2:  The second visualization for the temperature data is a **Chloropleth Map** that shows the Average Temperature per Province across all of the decades from 1960s to 2020s.
#### Interactions:
- Across the Decades Animation

<em>Note: the reason that only four decades are used (1960s, 1980s, 2000s, and 2020s) is because using more decades/dimensions would make the load time of the figure significantly higher reaching up to an hour. This visualization takes approximately 2-3 minutes to load in this notebook. </em>

In [ ]:
# def update_map_fig(n_intervals):
#     temp_sliced = temp_gdf[(temp_gdf['decade'].isin(['1960s', '1980s', '2000s', '2020s']) == True)].reset_index()
#     map_fig = px.choropleth_mapbox(temp_sliced,
#                                     geojson=temp_sliced.geometry,
#                                     locations=temp_sliced.index,
#                                     color='value',
#                                     color_continuous_scale='turbo',
#                                     range_color=[25, 32.5],
#                                     mapbox_style='streets',
#                                     zoom=5,
#                                     center={"lat": 12.8797, "lon": 121.7740},
#                                     opacity=0.6,
#                                     animation_frame="decade"
#                                     )
#     map_fig.update_layout(
#         coloraxis_colorbar=dict(title=f"Average<br>Temperature(°C)", yanchor="top", xanchor='left',
#                                 y=1, x=0, ticks="outside", ticklabelposition="outside left", thickness=10, title_font_color='#0c232c',
#                                 tickvals=[i for i in range(25, 33)],
#                                 tickmode='array',
#                                 ticksuffix='°C',
#                                 tickfont=dict(
#                                     size=12,
#                                     color='#0c232c')
#                                 )
#     )
#     map_fig.update_layout(
#         margin=dict(l=0, r=0, t=0, b=0),
#         updatemenus=[{
#             'buttons': [
#                 {'visible': True}
#             ],
#             'direction': 'left',
#             'pad': {'t': 0, 'b': 0, 'l': 0, 'r': 0},
#             'showactive': False,
#             'type': 'buttons',
#             'x': 0.15,
#             'xanchor': 'right',
#             'y': 0.21,
#             'yanchor': 'top'
#         }],
#         sliders=[{
#             'active': 0,
#             'x': 0.07,
#             'y': 0.19,
#             'len': 0.5,
#             'xanchor': 'left',
#             'yanchor': 'top',
#             'currentvalue': {
#                 'font': {'size': 16},
#                 'prefix': 'Decade:',
#                 'visible': True,
#                 'xanchor': 'right'
#             },
#             'transition': {'duration': 1000, 'easing': 'cubic-in-out'},
#             'pad': {'t': 0, 'b': 0, 'l': 0, 'r': 0}
#         }]
#     )
#     hover_template = '<b>%{customdata[0]}</b><br>Average Temp: %{customdata[1]:.2f}°C<extra></extra>'
#     map_fig.update_traces(hovertemplate=hover_template,
#                            customdata=temp_sliced[['name', 'value']])
#     for frame in map_fig.frames:
#         frame.data[0].hovertemplate = hover_template
#         frame.data[0].customdata = temp_sliced[['name', 'value']]

#     return map_fig

# map_fig = update_map_fig(1)
# map_fig.show()

In [ ]:
def update_map_fig(decade_value):
    map_fig = px.choropleth_mapbox(temperature_gdf,
                                    geojson=temperature_gdf.geometry,
                                    locations=temperature_gdf.index,
                                    color=decade_value,
                                    color_continuous_scale='turbo',
                                    range_color=[25, 33],
                                    mapbox_style='streets',
                                    zoom=5,
                                    center={"lat": 12.8797, "lon": 122.7740},
                                    opacity=0.6,
                                    )
    map_fig.update_layout(
        coloraxis_colorbar=dict(title=f"{decade_value}<br>Average<br>Temperature(°C)", yanchor="top", xanchor='left',
                                y=1, x=0, ticks="outside", ticklabelposition="outside left", thickness=10, title_font_color='#0c232c',
                                tickvals=[i for i in range(25, 33)],
                                tickmode='array',
                                ticksuffix='°C',
                                tickfont=dict(
                                    size=12,
                                    color='#0c232c')
                                )
    )
    map_fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
    )
    hover_template = '<b>%{customdata[0]}</b><br>during the '+ decade_value +'<br>Average Temp: %{customdata[1]:.2f}°C<extra></extra>'
    map_fig.update_traces(hovertemplate=hover_template,
                           customdata=temperature_gdf[['name', decade_value]])

    return map_fig

map_fig = update_map_fig('1970s')
map_fig.show()

**APP IDEAS**
- The temperature page will be in two columns 3:9
- The right one is the map TEMP 2
- The left one is a header and story explaining the map
- below the text will be a modal button "See More" that opens a modal
- The modal contains two columns, the left one is a text header and story, the right one is TEMP 1 bar graph
- The app will have a footer that has two buttons at its ends, "< BACK" and "NEXT >"
- Back will lead to the initial page and Next will lead to the Biodiversity Page

# Biodiversity

In [ ]:
print(biodiversity_gdf.shape)
biodiversity_gdf.head()

### BIO 1: The first visualization for the biodiversity data is a **Chloropleth Map** that shows the number of [Total/CR/EN/VU] unique species per Province.
#### Interactions:
- DropDown/RadioButtons that switch between "Total, CR, EN and VU" through Dash

In [ ]:
land_gdf = biodiversity_gdf[(biodiversity_gdf['area_type'].isin(['Land']))]

fig = px.choropleth_mapbox(land_gdf,
                            geojson=land_gdf.geometry,
                            locations=land_gdf.index,
                            color='Critically Endangered', # Change this based on dropdown
                            color_continuous_scale='balance',
                            range_color=[land_gdf['Critically Endangered'].min(), land_gdf['Critically Endangered'].max()],  
                            mapbox_style='streets',  
                            zoom=5,  
                            center={"lat": 12.8797, "lon": 122.7740}, 
                            opacity=0.6,
                            )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(coloraxis_colorbar=dict(title="Critically Endangered<br>Species",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',  
                            ticksuffix=' Species',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

hover_template = '<b>%{customdata[0]}</b><br>Unique Species: %{customdata[1]:.0f}<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=land_gdf[['name', 'Critically Endangered']])

fig.show()

### BIO 2: The second visualization for the biodiversity data is a DRILLDOWN operation on the previous **Chloropleth Map** that shows the number of [Total/CR/EN/VU] unique species per Province. In addition to this, the drilldown will be based on Island Groups
#### Interactions:
- A button that triggers the drilldown
- DropDown/RadioButtons that switch between "Total, CR, EN and VU" (from the previous map BIO 1)
- DropDown/RadioButtons that switch between "Luzon, Visayas, and Mindanao"

In [ ]:
# Set Island Group
land_gdf = biodiversity_gdf[(biodiversity_gdf['area_type'].isin(['Land'])) & (biodiversity_gdf['island_group'].isin(['Luzon']))]

# Createe the figure
fig = px.choropleth_mapbox(land_gdf,
                            geojson=land_gdf.geometry,
                            locations=land_gdf.index,
                            color='Critically Endangered', # Change this based on dropdown
                            color_continuous_scale='balance',
                            range_color=[land_gdf['Critically Endangered'].min(), land_gdf['Critically Endangered'].max()],  
                            mapbox_style='streets',  
                            zoom=6,  
                            center={"lat": 12.8797, "lon": 122.7740}, 
                            opacity=0.6,
                            )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(coloraxis_colorbar=dict(title="Critically Endangered<br>Species",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',  
                            ticksuffix=' Species',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

hover_template = '<b>%{customdata[0]}</b><br>Unique Species: %{customdata[1]:.0f}<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=land_gdf[['name', 'Critically Endangered']])

fig.show()

### BIO 3: The third visualization for the biodiversity data is ALSO a DRILLDOWN operation on the first chloropleth map and is a **Horizontal Stacked Bar Chart** that shows the number of [CR/EN/VU] unique species per Province. In addition to this, the drilldown will be based on Island Groups
#### Interactions:
- A button that triggers the drilldown (same button as BIO 2)
- DropDown/RadioButtons that switch between "Luzon, Visayas, and Mindanao" (same button as BIO 2)

In [ ]:
# Set Island Group
land_gdf = biodiversity_gdf[(biodiversity_gdf['area_type'].isin(['Land'])) & (biodiversity_gdf['island_group'].isin(['Luzon']))].sort_values(by='total_species', ascending=True)

# Create stacked bar plot using Plotly Express
fig = px.bar(land_gdf,
             x=['Vulnerable', 'Endangered', 'Critically Endangered'],  
             y='name',  
             title='Number of Unique Endangered Species in Luzon',
             labels={'Vulnerable', 'Endangered', 'Critically Endangered'},  
             orientation='h',  
             height=600,  
             color_discrete_sequence=['yellow', 'orange', 'red'], 
             )

fig.update_layout(
    xaxis_title='Number of Unique Species', 
    yaxis_title='Province',
    legend_title='Category',
)

hover_template = '<b>%{customdata[0]}</b><br>Unique Species: %{x}<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=land_gdf[['name']])

fig.show()

### BIO 4: The fourth visualization for the biodiversity data is a **Chloropleth Map** that shows the number of [Total/CR/EN/VU] unique species per sea.
#### Interactions:
- DropDown/RadioButtons that switch between "Total, CR, EN and VU" through Dash

In [ ]:
sea_gdf = biodiversity_gdf[(biodiversity_gdf['area_type'].isin(['Sea']))]

fig = px.choropleth_mapbox(sea_gdf,
                            geojson=sea_gdf.geometry,
                            locations=sea_gdf.index,
                            color='Critically Endangered', # Change this based on dropdown
                            color_continuous_scale='sunsetdark',
                            range_color=[sea_gdf['Critically Endangered'].min(), sea_gdf['Critically Endangered'].max()],  
                            mapbox_style='streets',  
                            zoom=5,  
                            center={"lat": 12.8797, "lon": 122.7740}, 
                            opacity=0.6,
                            )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(coloraxis_colorbar=dict(title="Critically Endangered<br>Species",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',  
                            ticksuffix=' Species',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

hover_template = '<b>%{customdata[0]}</b><br>Unique Species: %{customdata[1]:.0f}<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=sea_gdf[['name', 'Critically Endangered']])

fig.show()

### BIO 5: The fifth visualization for the biodiversity data is a **Horizontal Stacked Bar Chart** that shows the number of [CR/EN/VU] unique species per Sea.
#### Interactions:
- A button that shows the visualization

In [ ]:
sea_gdf = biodiversity_gdf[(biodiversity_gdf['area_type'].isin(['Sea']))]
# Create stacked bar plot using Plotly Express
fig = px.bar(sea_gdf,
             x=['Vulnerable', 'Endangered', 'Critically Endangered'],  
             y='name',  
             title='Number of Unique Endangered Species in Sea',
             labels={'Vulnerable', 'Endangered', 'Critically Endangered'},  
             orientation='h',  
             height=600,  
             color_discrete_sequence=['yellow', 'orange', 'red'], 
             )

fig.update_layout(
    xaxis_title='Number of Unique Species', 
    yaxis_title='Province',
    legend_title='Category',
)

hover_template = '<b>%{customdata[0]}</b><br>Unique Species: %{x}<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=sea_gdf[['name']])

fig.show()

**APP IDEAS**
- There is toggle that switches the page to show Land Data or Sea Data
- Bio 1, Bio 2, and Bio 3 is part of the Land and will have interactions that are connected to each other
- Land data will somehow be related to the temp data
- Bio 4, and Bio 5 is part of the Sea and will havee interactions that are also connected to each other
- Will also be compared to temp data

# Disaster

In [ ]:
print(disaster_gdf.shape)
disaster_gdf.head()

### DISASTER 1: The first visualization for the disaster data is a **Chloropleth Map** that shows the number of [Disaster Type] per Province.
#### Interactions:
- DropDown/RadioButtons that switch between Disaster Types (Total, Storm, Flood, Earthquake, Volcanic, Mass Movement, Drought) through Dash

In [ ]:
fig = px.choropleth_mapbox(disaster_gdf,
                            geojson=disaster_gdf.geometry,
                            locations=disaster_gdf.index,
                            color='Total Disaster Count', # Change based on dropdown value
                            color_continuous_scale='reds',
                            range_color=[disaster_gdf['Total Disaster Count'].min(), disaster_gdf['Total Disaster Count'].max()],  
                            mapbox_style='streets',  
                            zoom=5,  
                            center={"lat": disaster_gdf.geometry.centroid.y.mean(), 
                                    "lon": disaster_gdf.geometry.centroid.x.mean()}, 
                            opacity=0.6,
                            )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(coloraxis_colorbar=dict(title="Total Disaster<br>Count",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',  
                            # ticksuffix=' disasters',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

hover_template = '<b>%{customdata[0]}</b><br>%{customdata[1]:.0f} disasters<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=disaster_gdf[['Area Name', 'Total Disaster Count']])

fig.show()

### DISASTER 2: The second visualization for the disaster data is a **Horizontal Bar Chart** that shows the number of [Disaster Type] per Province per [Island Group].
#### Interactions:
- DropDown/RadioButtons that switch between Disaster Types (Total, Storm, Flood, Earthquake, Volcanic, Mass Movement, Drought) through Dash (Same button as Disaster 1 visualization)
- DropDown/RadioButtons that switch between Island Groups (Luzon Visayas Mindanao)

In [ ]:
# Island Group
luzon_disaster = disaster_gdf[disaster_gdf['Island Group'] == 'Luzon'].sort_values(by='Total Disaster Count', ascending=True)

# Create stacked bar plot using Plotly Express
fig = px.bar(luzon_disaster,
             x='Total Disaster Count',  
             y='Area Name',  
             title='Total Disasters in Luzon per Province',
             orientation='h',  
             height=600,  
             color_discrete_sequence=['lightblue'], 
             )

fig.update_layout(
    xaxis_title='Total Disasters', 
    yaxis_title='Province',
)

hover_template = '<b>%{customdata[0]}</b><br>%{x} disasters<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=luzon_disaster[['Area Name']])

fig.show()

### DISASTER 3: The third visualization for the disaster data is a **Chloropleth Map** that shows the number of [Disaster Type] per REGION.
#### Interactions:
- DropDown/RadioButtons that switch between Disaster Types (Total, Storm, Flood, Earthquake, Volcanic, Mass Movement, Drought) through Dash

In [ ]:
Region_gdf = disaster_gdf.copy()
Region_tot_ave = Region_gdf.drop(columns=['geometry']).groupby('Region').sum()

def region_count(disaster_type,col_name):
    Region_gdf[col_name] = np.nan #Create a new column that contains the number of disasters per region

    for i1,r1 in Region_gdf.iterrows():
        for i2,r2 in Region_tot_ave.iterrows():
            if r1['Region'] == i2:
                Region_gdf.at[i1, col_name] = r2[disaster_type]
            else:
                continue

# Total Disasters per Region
region_count('Total Disaster Count','Region_tot')
# Storm per Region
region_count('Storm Count','Region_storm')
# Flood per Region
region_count('Flood Count','Region_flood')
# Earthquake per Region
region_count('Earthquake Count','Region_earth')
# Volcanic Activity per Region
region_count('Volcanic Activity Count','Region_vol')
# Mass Movement per Region
region_count('Mass Movement Count','Region_mass')
# Drought per Region
region_count('Drought Count','Region_drought')
Region_gdf.head()

In [ ]:
fig = px.choropleth_mapbox(Region_gdf,
                            geojson=Region_gdf.geometry,
                            locations=Region_gdf.index,
                            color='Region_tot', # Change based on dropdown value
                            color_continuous_scale='amp',
                            range_color=[Region_gdf['Region_tot'].min(), Region_gdf['Region_tot'].max()],  
                            mapbox_style='streets',  
                            zoom=5,  
                            center={"lat": Region_gdf.geometry.centroid.y.mean(), 
                                    "lon": Region_gdf.geometry.centroid.x.mean()}, 
                            opacity=0.6,
                            )

fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.update_layout(coloraxis_colorbar=dict(title="Total Disaster<br>Count",yanchor="top",xanchor='right',
                            y=1, x=1, ticks="outside", thickness=10, title_font_color='#0c232c',  
                            # ticksuffix=' disasters',
                            tickfont=dict(
                            size=12,
                            color='#0c232c')
                        )
                    )

hover_template = '<b>%{customdata[0]}</b><br>%{customdata[1]:.0f} disasters<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=Region_gdf[['Region', 'Region_tot']])

fig.show()

### DISASTER 4: The fourth visualization for the disaster data is a **Horizontal Bar Chart** that shows the number of [Disaster Type] per REGION per [Island Group].
#### Interactions:
- DropDown/RadioButtons that switch between Disaster Types (Total, Storm, Flood, Earthquake, Volcanic, Mass Movement, Drought) through Dash (Same button as Disaster 3 visualization)
- DropDown/RadioButtons that switch between Island Groups (Luzon Visayas Mindanao)

In [ ]:
# Island Group
luzon_reg_disaster = Region_gdf[Region_gdf['Island Group'] == 'Luzon'].groupby('Region')['Region_tot'].sum().sort_values(ascending=True)

# Create stacked bar plot using Plotly Express
fig = px.bar(x=luzon_reg_disaster.values,  
             y=luzon_reg_disaster.index,  
             title='Total Disasters in Luzon per Region',
             orientation='h',  
             height=600,  
             color_discrete_sequence=['blue'], 
             )

fig.update_layout(
    xaxis_title='Total Disasters', 
    yaxis_title='Region',
)

hover_template = '<b>%{y}</b><br>%{x} disasters<extra></extra>'

fig.update_traces(hovertemplate=hover_template,
                  customdata=luzon_reg_disaster.reset_index().values)

fig.show()


**APP IDEAS**
- There is toggle that switches the visualizations to rollup or drilldown between visualizing by province or by region
- Disaster 1 and 2 interactions will be connected
- Disasteer 3 and 4 interactions will be connected
- Header and stories to be provided as text 